In [36]:
# page4

import pandas as pd
from tqdm import tqdm
import time
import re

test = pd.read_csv('test.csv')

test['year'] = test['transaction_year_month'].astype(str).str[:4].astype(int)
test['month'] = test['transaction_year_month'].astype(str).str[4:].astype(int)

test.loc[(test['transaction_date'] == '21~30') | (test['transaction_date']=='21~28')| (test['transaction_date']=='21~29'), 'transaction_date'] = '21~31'

test.loc[(test['transaction_date'] == '1~10'), 'transaction_date'] = '01~10'
test['age'] = test['year'] - test['year_of_completion']

test.loc[test['age'] < 20, 'reconstruction'] = 'NO'
test.loc[test['age'] >= 20, 'reconstruction'] = 'YES'

regex = "\(.*\)"

for i in tqdm(range(len(test))):
  test.at[i,'apt'] = re.sub(regex, '', test.at[i,'apt'])

test['apt'].unique()

100%|██████████| 5463/5463 [00:00<00:00, 15952.45it/s]


array(['청학', '고관맨션', '우림그린', ..., '위너스', '동양드림', '센텀그린타워'], dtype=object)

In [37]:
# page91

test['apt'].value_counts()[:20]

apt_brand_list = test['apt'].value_counts()[:20].index

test['branded'] = False

for brand in tqdm(apt_brand_list):
  #print(f"brand[{brand}]")
  if brand != '' :
    loc_list = test['apt'].str.contains(brand)

    test['apt'].loc[loc_list] = brand
    test['branded'].loc[loc_list] = True

test['apt'].loc[~test['branded']] = 'others'

test['apt'].value_counts()

  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipython-input-37-3122903839.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['apt'].loc[loc_list] = brand
/tmp/ipython-input-37-3122903839.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and

,count
apt,
others,4319
현대,299
삼성,130
동아,97
우성,93
한신,83
대림,78
두산,71
벽산,67


In [38]:
test['branded'].value_counts()

,count
branded,
False,4319
True,1144


In [39]:
"""
동원로얄듀크     934.779456     0
벽산        1157.018489       1
삼환        1309.523928       2
코오롱       1354.262951      3
대우        1416.341592      4
한신        1485.017371      5
극동        1502.618613      6
대림        1537.601553      7
쌍용        1567.364139      8
삼익        1585.292149      9
경남        1592.788764      10
others    1594.194121       11
롯데캐슬      1609.534382     12
현대        1635.864649      13
두산        1655.227154      14
신동아       1704.860704     15
우성        1791.239807      16
한양        1806.277905      17
주공2       1861.882436      18
삼성        1931.917454      19
"""

# others=11
apt_dict = {"현대":'13', "삼성":'19', "동아":'15', "우성":'16', "한신":'5', "대림":'7', "두산":'14', "벽산":'1', "대우":'4', "극동":'6', "삼환":'2'}

for i in tqdm(range(len(test))):
  if test.at[i,'apt'] in apt_dict:
    test.at[i,'apt'] = apt_dict[test.at[i,'apt']]
  else:
    test.at[i,'apt'] = '11'

test.loc[:,'apt'] = test.loc[:,'apt'].astype('int')
test['apt'].value_counts()

100%|██████████| 5463/5463 [00:00<00:00, 15973.06it/s]


,count
apt,
11,4441
13,299
19,130
15,97
16,93
5,83
7,78
14,71
1,67


In [41]:
test_X_cat = test[['city','transaction_date','reconstruction']]

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


le_arr = {}

le = LabelEncoder()
le_arr['city'] = le
test_X_cat.loc[:,'city'] = le.fit_transform(test_X_cat.loc[:,'city'])

le = LabelEncoder()
le_arr['transaction_date'] = le
test_X_cat.loc[:,'transaction_date'] = le.fit_transform(test_X_cat.loc[:,'transaction_date'])

le = LabelEncoder()
le_arr['reconstruction'] = le
test_X_cat.loc[:,'reconstruction'] = le.fit_transform(test_X_cat.loc[:,'reconstruction'])

test_X_num = test[['exclusive_use_area','floor','age']]
test_X_num.head()

mm_scaler = MinMaxScaler()

test_minmax = mm_scaler.fit_transform(test_X_num)

test_minmax = pd.DataFrame(test_minmax, index=test_X_num.index, columns=test_X_num.columns)

X = pd.concat([test_minmax, test_X_cat, test['apt']], axis=1)
#y = test['transaction_real_price']

X.head()

,exclusive_use_area,floor,age,city,transaction_date,reconstruction,apt
0,0.095730,0.053571,0.267857,1,1,0,11
1,0.221594,0.053571,0.714286,0,2,1,11
2,0.236089,0.053571,0.500000,0,1,1,11
3,0.384075,0.196429,0.232143,1,0,0,11
4,0.414810,0.392857,0.053571,0,1,0,11
